In [1]:
import json
import string
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

In [2]:
data_one = []
reviewdata_df = []

In [3]:
data_file = open('F:\yelp\yelp_academic_dataset_review.json')
length = 2225213  # I calculated length prior to this calculation
for i, line in enumerate(data_file):
    if i<(length/4):
        data_one.append(json.loads(line))
data_file.close()

In [4]:
df = pd.DataFrame(data_one)

In [5]:
df.shape

(556303, 8)

In [6]:
data_main = []
with open('F:\yelp\yelp_academic_dataset_user.json') as f:
    for line in f:
        data_main.append(json.loads(line))

In [7]:
df_user = pd.DataFrame(data_main)

In [8]:
def user_with_friends(x):
    if len(x['friends']) != 0:
        return x

In [9]:
df_user_with_friends = df_user.apply(user_with_friends, axis=1)

In [10]:
df_user_with_friends = df_user_with_friends.dropna()

In [11]:
df_user.shape


(552339, 11)

In [12]:
df_user_with_friends.shape

(249440, 11)

In [13]:
merged_df = pd.merge(df, df_user_with_friends, on='user_id', how='inner')

In [14]:
merged_df.head(n=1)

,business_id,date,review_id,stars,text,type_x,user_id,votes_x,average_stars,compliments,elite,fans,friends,name,review_count,type_y,votes_y,yelping_since
0,UsFtqoBl7naz8AVUBZMjQQ,2013-11-08,Di3exaUCFNw1V4kSNW5pgA,5,All the food is great here. But the best thing...,review,uK8tzraOp4M5u3uYrqIBXg,"{u'funny': 0, u'useful': 0, u'cool': 0}",3.84,{},[],2,"[53wSMIztehsjP-vJZBS54A, CXhOU5apSRSTtEUiyoqH3Q]",Robin,46,user,"{u'funny': 2, u'useful': 36, u'cool': 5}",2013-05


In [15]:
merged_df.shape

(394815, 18)

In [16]:
merged_df.stars.value_counts(ascending=True)

2     36085
1     38347
3     63586
4    124241
5    132556
Name: stars, dtype: int64

In [17]:
data_business = []
with open('F:\yelp\yelp_academic_dataset_business.json') as f:
    for line in f:
        data_business.append(json.loads(line))

In [18]:
df_business = pd.DataFrame(data_business)

In [19]:
business_merged_df = pd.merge(merged_df, df_business, on='business_id', how='inner')

In [20]:
business_merged_df.shape

(394815, 32)

In [21]:
business_merged_df.drop(['neighborhoods', 'latitude', 'longitude', 'elite', 'yelping_since', 'compliments', 'full_address', 'city', 'state',
                         'open', 'hours', 'attributes'], axis=1, inplace=True)

In [22]:
business_merged_df.head(n=1)

,business_id,date,review_id,stars_x,text,type_x,user_id,votes_x,average_stars,fans,friends,name_x,review_count_x,type_y,votes_y,categories,name_y,review_count_y,stars_y,type
0,UsFtqoBl7naz8AVUBZMjQQ,2013-11-08,Di3exaUCFNw1V4kSNW5pgA,5,All the food is great here. But the best thing...,review,uK8tzraOp4M5u3uYrqIBXg,"{u'funny': 0, u'useful': 0, u'cool': 0}",3.84,2,"[53wSMIztehsjP-vJZBS54A, CXhOU5apSRSTtEUiyoqH3Q]",Robin,46,user,"{u'funny': 2, u'useful': 36, u'cool': 5}",[Nightlife],Clancy's Pub,4,3.5,business


In [24]:
%timeit business_sorted_df = business_merged_df.sort_index(by=['business_id'], ascending=[True])

1 loops, best of 3: 1.34 s per loop


F:\PythonIDE\lib\site-packages\ipykernel\__main__.py:257: FutureWarning: by argument to sort_index is deprecated, pls use .sort_values(by=...)


In [23]:
def foodie_businesses(x):
    if 'Food' in x or 'Restaurants' in x:
        return x

In [31]:
for row in business_merged_df.itertuples():
    if 'Food' in row[16] or 'Restaurants' in row[16]:
        business_merged_df['desired_category'] = "foodie"

KeyboardInterrupt: 

In [40]:
grouped = business_merged_df.head(n=500).groupby(business_merged_df['business_id'])

In [33]:
def foodie_business(group, biz_id):
    for row in group.itertuples():
        if 'Food' in row[16] or 'Restaurants' in row[16]:
            return "foodie"

In [61]:
business_merged_df.dtypes

business_id          object
date                 object
review_id            object
stars_x               int64
text                 object
type_x               object
user_id              object
votes_x              object
average_stars       float64
fans                float64
friends              object
name_x               object
review_count_x      float64
type_y               object
votes_y              object
categories           object
name_y               object
review_count_y        int64
stars_y             float64
type                 object
desired_category     object
dtype: object

In [72]:
def func(x):
    if 'Food' in x['categories'] or 'Restaurants' in x['categories']:
        print x
        return x

In [75]:
grouped = business_merged_df.head(n=500).groupby(business_merged_df['business_id']).apply(func)

TypeError: func() got an unexpected keyword argument 'axis'

In [74]:
grouped

""


In [71]:
for business_id, group in grouped:
    print group
    #     for cat in group.categories:
#         print cat

In [62]:
foodie_businesses = {}
for business_id, group in grouped:
    for cat in group.categories:
        if 'Food' in cat or 'Restaurants' in cat:
            foodie_businesses.update({
                    "business_id": business_id,
                    "date": group.date,
                    "review_id": group.review_id,
                    "stars_x": group.stars_x,
                    "text": group.text,
                    "type_x": group.type_x,
                    "user_id": group.user_id,
                    "votes_x": group.votes_x,
                    "average_stars": group.average_stars,
                    "fans": group.fans,
                    "friends": group.friends,
                    "name_x": group.name_x,
                    "review_count_x": group.review_count_x,
                    "type_y": group.type_y,
                    "votes_y": group.votes_y,
                    "categories": group.categories,
                    "name_y": group.name_y,
                    "review_count_y": group.review_count_y,
                    "stars_y": group.stars_y,
                    "type": group.type,
                    "desired_category": cat
                    
                    
                })
#            print business_id
#            print cat
        
#         if 'Food' in cat or 'Restaurants' in cat:
#             business_merged_df['desired_category'] = "foodie"
        
#     if 'Food' in group['categories'] or 'Restaurants' in group['categories']:
#         print "foodie"
    #business_merged_df['desired_category'] = foodie_business(group, business_id)

In [65]:
foodie_businesses

{'average_stars': 3     3.84
 4     4.00
 5     3.40
 6     2.92
 7     3.52
 8     4.26
 9     3.92
 10    4.24
 11    5.00
 Name: average_stars, dtype: float64,
 'business_id': u'mVHrayjG3uZ_RLHkLj-AMg',
 'categories': 3     [Bars, American (New), Nightlife, Lounges, Res...
 4     [Bars, American (New), Nightlife, Lounges, Res...
 5     [Bars, American (New), Nightlife, Lounges, Res...
 6     [Bars, American (New), Nightlife, Lounges, Res...
 7     [Bars, American (New), Nightlife, Lounges, Res...
 8     [Bars, American (New), Nightlife, Lounges, Res...
 9     [Bars, American (New), Nightlife, Lounges, Res...
 10    [Bars, American (New), Nightlife, Lounges, Res...
 11    [Bars, American (New), Nightlife, Lounges, Res...
 Name: categories, dtype: object,
 'date': 3     2013-10-20
 4     2013-03-15
 5     2013-03-30
 6     2013-11-07
 7     2014-11-03
 8     2014-12-18
 9     2015-03-06
 10    2015-03-08
 11    2015-12-04
 Name: date, dtype: object,
 'desired_category': [u'Bars',
  u'

In [64]:
business_merged_df = pd.DataFrame.from_dict(foodie_businesses, orient='index')

AttributeError: 'unicode' object has no attribute 'items'

In [55]:
for row in business_merged_df.itertuples():
    if row[1] == 'UsFtqoBl7naz8AVUBZMjQQ':
        print row[16]

[u'Nightlife']
[u'Nightlife']
[u'Nightlife']


In [76]:
business_merged_df = business_merged_df.loc[business_merged_df.categories.apply(lambda cat: 'Food' in cat or 'Restaurants' in cat)]

In [79]:
business_merged_df.shape

(252836, 21)

In [ ]:
business_merged_df = business_merged_df.apply(foodie_businesses, axis=1)

In [ ]:
for row in business_merged_df.itertuples():
    if 

In [24]:
business_merged_df = business_merged_df.apply(lambda row: foodie_business(row[16]), axis=1)

NameError: ("global name 'foodie_business' is not defined", u'occurred at index 0')

In [30]:
business_merged_df['desired_categories'] = business_merged_df['categories'].apply(lambda x: 'foodie' if x != 0)

SyntaxError: invalid syntax (<ipython-input-30-09192ae75c10>, line 1)

In [36]:
business_merged_df.shape

(394815, 20)